### MOTION DETECTION USING FRAME DIFFERENCE

In [ ]:
# import libraries
import cv2 # help: https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html

In [ ]:
def detect_motion(previous_frame, current_frame, next_frame, threshold = 40):
    """Detects motion by finding frame difference across 3 consercutive frames

    Args:
        previous_frame (Mat): The previous frame
        current_frame (Mat): The current frame
        next_frame (Mat): The next frame
        threshold (int): The level of sensitivity
    """
    # convert all the frames to grayscale
    next_frame = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
    current_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    previous_frame = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)
    
    # set frames to same size
    next_frame = cv2.resize(next_frame, (200, 200))
    current_frame = cv2.resize(current_frame, (200, 200))
    previous_frame = cv2.resize(previous_frame, (200, 200))
    
    # get the differences between the frames and use bitwise and operator to get the resolved differences
    # between three successive frames
    next_cur_diff = cv2.absdiff(next_frame, current_frame)
    cur_prev_diff = cv2.absdiff(current_frame, previous_frame)
    resolve_diff = cv2.bitwise_and(next_cur_diff, cur_prev_diff)
    
    # get the difference count and use to detect motion
    if (resolve_diff > 100 - threshold).sum() > 50:
        print("motion detected")
        
    # return frame difference
    return resolve_diff

In [ ]:
# start video capture
capture = cv2.VideoCapture(0)

# check whether the capture was opened successfully
if not capture.isOpened():
    print("Unable to start camera")

# initialize frame tracker and frame holders
frames_tracker = 0
previous_frame, current_frame, next_frame = None, None, None

while True:
    # capture the next frame
    ret, frame = capture.read()
    
    # check whether the frame is available or not
    if frame is None:
        break
    
    if frames_tracker == 0:
        # grab first frame
        previous_frame = frame.copy()
        frames_tracker += 1
    elif frames_tracker == 1:
        # grab second frame
        current_frame = frame.copy()
        frames_tracker += 1
    elif frames_tracker == 2:
        # grab third frame
        next_frame = frame.copy()
        frames_tracker += 1
    elif frames_tracker == 3:
        # detect motion
        frame = detect_motion(previous_frame, current_frame, next_frame)
        frames_tracker += 1
    else:
        # reshuffle frames
        previous_frame = current_frame
        current_frame = next_frame
        next_frame = frame
        # detect motion
        frame = detect_motion(previous_frame, current_frame, next_frame)
    
    # show the captured frame
    cv2.imshow("Video", frame)
    
    # wait 30 milliseconds for a key press event
    keypressed = cv2.waitKey(30)
    # if the ESC key is pressed, destroy all windows and release resources
    if keypressed == 27:
        # release camera
        capture.release()
        # destroy all windows
        cv2.destroyAllWindows()
        break